# Calculator

## 1. 기능 구현

In [2]:
import os
import json
import numpy as np


# 함수.1
def extract_flattened_keypoints_from_json(file_path: str):
    """
    JSON 파일에서 Keypoints 값을 추출하고 평탄화하여 벡터로 변환하는 함수
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    keypoints = data.get("keypoints", [])
    flattened = [flattenend_data if flattenend_data is not None else 0 for kp in keypoints for flattenend_data in (kp['x'], kp['y'])]

    return flattened

# 함수.2
def convert_vectors_by_flattened(directory_path: str):
    """
    directory내 모든 JSON 파일에서 좌표 벡터를 수집하여 리스트로 반환하는 함수
    """
    # 저장소 만들기
    vectors = []
    files = os.listdir(directory_path)

    # 디렉토리 안에 JSON 데이터만 불러오기
    for file in files:
        if file.endswith('.json'):
            file_path = os.path.join(directory_path, file)

            # 함수.1을 이용해 모든 파일의 좌표값을 flattened
            coords = extract_flattened_keypoints_from_json(file_path)

            if len(coords) == 34:
                vectors.append(np.array(coords))

    return vectors

# 함수.3
def cosine_similarity(vec1, vec2):
    """두 개의 벡터(vec1, vec2)의 코사인 유사도를 계산"""
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    
    # 벡터의 크기(Norm) 계산
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    
    # 두 벡터의 내적(dot product) 계산 후 코사인 유사도 반환
    similarity = np.dot(vec1, vec2) / (norm1 * norm2)
    
    return similarity

# 함수. 4
def compare_cosine_similarity(result1, result2):
    """
    result1과 result2의 코사인 유사도를 비교하는 함수
    result1과 result2는 각각 다차원 리스트(2D 배열) 형태일 수 있음.
    """
    similarities = []
    
    # 각 벡터를 1:1로 비교 (result1과 result2의 길이가 같다고 가정)
    for vec1, vec2 in zip(result1, result2):
        sim = cosine_similarity(vec1, vec2)
        similarities.append(sim)

    return similarities

# 함수. 5
def compare_cosine_similarity_matrix(answer_vectors, realtime_vectors):
    """
    정답 벡터 리스트와 실시간 벡터 리스트 간 모든 코사인 유사도를 비교

    Parameters:
    - answer_vectors: List[np.ndarray]  → 정답 데이터 벡터 리스트 (예: 87개)
    - realtime_vectors: List[np.ndarray]  → 실시간 데이터 벡터 리스트 (예: N개)

    Returns:
    - 2차원 리스트 (87 × N): 각 정답 벡터와 실시간 벡터 간 코사인 유사도 행렬
    """
    similarity_matrix = []

    for a_vec in answer_vectors:
        row = []
        for r_vec in realtime_vectors:
            # 벡터 유사도 계산 (예외 방지)
            norm1 = np.linalg.norm(a_vec)
            norm2 = np.linalg.norm(r_vec)
            if norm1 == 0 or norm2 == 0:
                sim = 0
            else:
                sim = np.dot(a_vec, r_vec) / (norm1 * norm2)
            row.append(sim)
        similarity_matrix.append(row)

    return similarity_matrix

### Test

In [3]:
answer_directory_path1 = '../../LJH/data/front_json'
vec1 = convert_vectors_by_flattened(answer_directory_path1)
# print(vec1)

In [4]:
realtime_directory_path = '../../LJH/data/side_json'
vec2 = convert_vectors_by_flattened(realtime_directory_path)
# print(vec2)

In [5]:
print(vec1[0])
print(vec2[0])

[643 143 651 136 634 136 663 147 621 148 678 199 605 199 701 222 585 217
 668 216 621 213 670 331 618 332 685 438 604 440 697 544 592 546]
[646 144 654 137   0   0 675 146   0   0 666 195 682 195 607 217   0   0
 526 216   0   0 662 335 672 333 660 441 663 438 661 546 662 538]


In [6]:
# 코사인 유사도 분석 (각 이미지 비교: 87장)
cosines_result = compare_cosine_similarity(vec1, vec2)
print(cosines_result)
print(len(cosines_result))

[0.8962352283040476, 0.8964174912829719, 0.8959659013152648, 0.8962713511968137, 0.8963241246274384, 0.8961436861055866, 0.8959218730413666, 0.8959082428297367, 0.8957897710054379, 0.8957448760612732, 0.8957805907508141, 0.8954975593363913, 0.896396933007755, 0.8951436991138138, 0.8948434776304776, 0.8949080441415501, 0.8946556322773294, 0.8943560362276349, 0.8944429048580749, 0.8939502865041099, 0.8939287188802071, 0.8938800351509506, 0.8936563413919987, 0.8964020073664912, 0.8930766875671309, 0.8927414117836215, 0.8924904810880937, 0.8924536608397474, 0.8924494718969014, 0.8923844402204929, 0.8919913124807987, 0.891953352993941, 0.8918869068400367, 0.89162878165455, 0.8964069104094542, 0.8914860388561506, 0.8910487408431579, 0.8910340118100843, 0.8912052294982855, 0.8911189248767701, 0.8907611151465794, 0.8905753895799267, 0.8905774021543454, 0.8904020966063688, 0.8903497153662492, 0.8962910404703108, 0.8902912528208135, 0.8902572948235125, 0.8902345469366837, 0.8904675872146717, 0.8

In [7]:
# 코사인 유사도 분석 (각 keypoints 하나의 값 모음)
cosines_result = compare_cosine_similarity_matrix(vec1, vec2)
print(cosines_result)
print(len(cosines_result[0]))

[[0.8962352283040476, 0.8963036509384239, 0.8961533733439775, 0.8960670544840255, 0.8960819677848528, 0.8960347308386359, 0.8958466261612695, 0.8958352378820239, 0.8957904049947457, 0.8957653983084843, 0.8956245326994909, 0.8952603062852411, 0.8962568323452131, 0.8948803919082489, 0.8945932467253577, 0.8945986890344737, 0.8942236141003246, 0.8938294562769636, 0.8934621988881727, 0.8928502489636787, 0.8928501576453539, 0.8925391557775555, 0.8920779008915191, 0.8962762273240118, 0.8914236554810976, 0.8909159036445752, 0.8907143187539944, 0.8900573265476793, 0.8894932852584575, 0.8890472294728666, 0.8886364970863011, 0.8885895660721861, 0.8881422879717098, 0.8877005231872337, 0.8962373664381893, 0.8872640462341439, 0.8869742516471969, 0.8869805748558215, 0.8866834193857435, 0.8865707374919441, 0.8863500210684043, 0.8862991870216238, 0.8863055865945411, 0.8862911955288352, 0.8862447590256833, 0.8961979308683183, 0.8862465820175001, 0.8862875329378813, 0.8862544177348913, 0.886293583025296,

In [8]:
# 정답 벡터 데이터 저장하기